In [63]:
import pandas as pd


In [64]:
train = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/train.csv')
test = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/test.csv')
movies = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/movies.csv', delimiter= '::', encoding= 'latin1')
users = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/users.csv', delimiter='::')

/var/folders/5j/7vfy0ndj133gm55cj1czw8rr0000gn/T/ipykernel_4811/2668559298.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/movies.csv', delimiter= '::', encoding= 'latin1')
/var/folders/5j/7vfy0ndj133gm55cj1czw8rr0000gn/T/ipykernel_4811/2668559298.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/users.csv', delimiter='::')


In [65]:
movies['year'] =  movies['Title'].str.extract(r'\((\d{4})\)', expand=False)
movies['Title'] = movies['Title'].str.replace(r'\(\d{4}\)$',"")
movies['Genres'] = movies['Genres'].apply(lambda x: x.split('|'))
movies.head()


/var/folders/5j/7vfy0ndj133gm55cj1czw8rr0000gn/T/ipykernel_4811/3856545431.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['Title'] = movies['Title'].str.replace(r'\(\d{4}\)$',"")


,movieID,Title,Genres,year
0,1,Toy Story,"[Animation, Children's, Comedy]",1995
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [66]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,796,F,1,10,48067
1,3203,M,56,16,70072
2,4387,M,25,15,55117
3,4771,M,45,7,02460
4,1191,M,25,20,55455


In [67]:
train.head()

,TrainDataID,UserID,MovieID,Rating
0,1,796,1193,5
1,2,796,661,3
2,3,796,914,3
3,4,796,3408,4
4,5,796,2355,5


In [68]:
train_full = train.merge(users, on = 'UserID').merge(movies, left_on = 'MovieID', right_on='movieID')
train_full = train_full.drop(['movieID', 'Title'], axis = 1)

In [69]:
train_full.head()

,TrainDataID,UserID,MovieID,Rating,Gender,Age,Occupation,Zip-code,Genres,year
0,1,796,1193,5,F,1,10,48067,[Drama],1975
1,112,3203,1193,5,M,56,16,70072,[Drama],1975
2,1207,1313,1193,4,M,25,12,32793,[Drama],1975
3,1373,5815,1193,4,M,25,7,22903,[Drama],1975
4,1579,3531,1193,5,M,50,1,95350,[Drama],1975


In [70]:
train_full.dtypes

TrainDataID     int64
UserID          int64
MovieID         int64
Rating          int64
Gender         object
Age             int64
Occupation      int64
Zip-code       object
Genres         object
year           object
dtype: object

In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizeTra, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer

In [72]:
cat_cols = ['UserID', 'MovieID', 'Gender', 'Occupation', 'Zip-code']
num_cols = ['Age', 'year']
lab_cols = ['Genres']

In [76]:
#chatgpt solution
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer

class MultiLabelBinarizerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()
    
    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self
    
    def transform(self, X):
        return self.mlb.transform(X)
    
    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)


In [84]:
encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=  False)
scaler = StandardScaler()
labs = MultiLabelBinarizerTransformer()

mod = RandomForestClassifier()

pre = make_column_transformer((encoder, cat_cols),
                              (scaler, num_cols),
                              (labs, lab_cols), 
                              remainder= 'passthrough')

pipeline = Pipeline([('encoder', pre),
                     ('model', mod)])

In [81]:
y = train_full['Rating']
X = train_full.drop(['Rating', 'TrainDataID'], axis = 1)

In [ ]:
X_transformed = pre.fit_transform(X)

In [ ]:
X_transformed.size

In [ ]:
pipeline.fit(X)